In [1]:
import pandas as pd
import numpy as np
from nltk.util import ngrams
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
np.random.seed(1322)

# HW3:  Классификация имен
В этом домашнем задании мы рассмотрим задачу бинарной классификации. Пусть дано два списка имен:  мужские и женские имена.  Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

## 0. Loading data

In [2]:
male_names = pd.read_csv('data/male.txt', header=-1, names=['name'], encoding = 'latin1')
female_names = pd.read_csv('data/female.txt', header=-1, names=['name'], encoding = 'latin1')

In [3]:
male_names_number = len(male_names)
female_names_number = len(female_names)
print('Количество мужских имен: ', male_names_number)
print('Количество женских имен: ', female_names_number)

Количество мужских имен:  2943
Количество женских имен:  5001


## 1. Filtering data

Предварительная обработка данных:<br>
* удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими одновременно), если такие есть.<br>
* создайте тестовое множество по следующему принципу: 20% от общего количества имен на каждую букву (т.е. 20% от имен на букву А, 20% имен на букву B, и.т.д.)

### 1.1 Looking for same names in both male and female df

In [4]:
same_names = female_names.merge(male_names, on=['name'], how='inner')
print('Количество совпадающих имен: ', len(same_names))
print('Список имен, являющихся одновременно и мужскими, и женскими: (первые 10)')
same_names.head(n = 10)

Количество совпадающих имен:  365
Список имен, являющихся одновременно и мужскими, и женскими: (первые 10)


,name
0,Abbey
1,Abbie
2,Abby
3,Addie
4,Adrian
5,Adrien
6,Ajay
7,Alex
8,Alexis
9,Alfie


### 1.2 And deleting duplicated names

In [5]:
remove_criterion = lambda row: row['name'] not in same_names['name'].values
male_names = male_names[male_names.apply(remove_criterion, axis=1)]
female_names = female_names[female_names.apply(remove_criterion, axis=1)]

In [6]:
print('Количество мужских имен после удаления дубликатов: ', len(male_names))
print('Количество женских имен после удаления дубликатов: ', len(female_names))
print('Все ок? ', (male_names_number - len(same_names) == len(male_names)) & 
      (female_names_number - len(same_names) == len(female_names)))

Количество мужских имен после удаления дубликатов:  2578
Количество женских имен после удаления дубликатов:  4636
Все ок?  True


### 1.3 Creating test sample (20% of dataset)

In [7]:
def createTrainTestSample(dataframe, letters_list, field, field2, test_percentage=0.2, rnd_state=0):
    train_X, train_y, test_X, test_y = ([] for i in range(4))
    for letter_value in letters_list:
        letter_df = dataframe[dataframe[field].str.startswith(letter_value)]
        X_train, X_test, y_train, y_test = train_test_split(letter_df[field].tolist(), letter_df[field2].tolist(), 
                                                            test_size=test_percentage, random_state=rnd_state)
        train_X.extend(X_train)
        test_X.extend(X_test)
        train_y.extend(y_train)
        test_y.extend(y_test)
    return train_X, test_X, train_y, test_y

In [8]:
def getListOfFirstLetters(dataframe, field):
    letters = dataframe[field].astype(str).str[0]
    letters = np.unique(letters.tolist())
    return letters

In [9]:
all_names = pd.concat([male_names, female_names], axis=0, ignore_index=True)
all_names['gender'] = np.concatenate((np.ones(len(male_names)), np.zeros(len(female_names))), axis=0).astype(int)
print('Общее количество имен: ', len(all_names))
print('Все ок? ', len(all_names) == len(male_names) + len(female_names))

Общее количество имен:  7214
Все ок?  True


In [10]:
# getting list of fisrt letters of names
lettes_list = getListOfFirstLetters(all_names, 'name')
print('Список первых букв имен, присутствующих в датасете:')
lettes_list

Список первых букв имен, присутствующих в датасете:


array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], 
      dtype='<U1')

In [11]:
%%time
X_train, X_test, y_train, y_test = createTrainTestSample(all_names, lettes_list, 'name', 'gender') 

CPU times: user 140 ms, sys: 4.68 ms, total: 145 ms
Wall time: 145 ms


## 2. Fitting Naive Bayes classifier

Используйте метод наивного Байеса для классификации имен: в качестве признаков используйте символьные n-граммы. <br>Сравните результаты, получаемые при разных n= 2, 3, 4 по F-мере и аккуратности. В каких случаях метод ошибается? <br>Для генерации n-грамм используйте <br><b>from nltk.util import ngrams.<b>

In [12]:
def remove_duplicates(list_to_filter):
    return list(set(list_to_filter))

In [13]:
def tuple_to_string(tuple_to_convert):
    return ''.join(tuple_to_convert)

In [14]:
def generateNgrams(dataframe, field, n):
    all_ngrams_list = []
    for name_value in dataframe[field].str.lower():
        for ngram in ngrams(name_value, n):
            all_ngrams_list.append(ngram)
    return remove_duplicates(all_ngrams_list)

In [15]:
def genFitMatrix(names_list, all_features):
    matrix = np.zeros((len(names_list), len(all_features)))
    for i, name_value in enumerate(names_list):
        for j, feature_value in enumerate(all_features):
            if tuple_to_string(feature_value) in name_value:
                matrix[i][j] = 1
    return matrix

In [16]:
list_ngrams = generateNgrams(all_names, 'name', 4)

In [17]:
X_train = genFitMatrix(X_train, list_ngrams)

In [18]:
X_test = genFitMatrix(X_test, list_ngrams)

In [19]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("accuracy =", accuracy_score(y_test, y_pred))
print("F-score =", f1_score(y_test, y_pred))

accuracy = 0.732326698696
F-score = 0.705882352941
